# 3 - Agentic Use Cases (Extra - 10 mins)

## What You Will Learn

In this exercise you will learn how to create domain-specific tools that can be used by an AI agent to address more complex information goals.

The GraphRAG Tool automatically creates domain-specific tools that can be consumed via the Model Context Protocol (see below). The ability to create domain-specific tools depends on two other features of the toolkit, introduced in previous exercises:

 - **Multi-tenancy:** The ability to host multiple separate lexical graphs within the same underlying graph and vector stores. This capability allows you to manage and query different sets of data within a shared infrastructure.
 - **Inferred schemas:** When indexing data, the GraphRAG Toolkit creates schema nodes that represent the implicit domain sematics at the entity-relationship tier of the lexical graph.
 
This exercise use two datasets, which have already been pre-built for you:

  - An aircraft information dataset sourced from Wikipedia
  - An aviation accident dataset courtesy of the National Transportation Safety Board (NTSB)
  
In the course of this exercise you will:
  
  - Review the inferred schemas for each of these two tenant graphs
  - Create an MCP server that automatically creates tools for these tenant graphs
  - Create an MCP client
  - Review the tool descriptions generated by the toolkit
  - Create an agent that orchestrates multiple question-answering interactions to help answer complex questions
  - Review the interactions between the agent and the tools
  

## MCP Tooling

The [Model Context Protocol](https://modelcontextprotocol.io/introduction) (MCP) is an open protocol that standardizes how applications provide context to LLMs.

In this example, you are going to create a 'catalog' of tools, one per tenant in a multi-tenant graph. Each tool is capable of answering domain-specific questions based on the data in its tenant graph. This catalog will be advertised to clients via an MCP server. Clients (typically agents and LLMs) can then browse the catalog and choose appropriate tools for addressing their information goals.

Each tool in the catalog is accompanied by an auto-generated description that helps a client understand the domain, scope, potential uses and kinds of questions covered by the tool. 

![Tools](./images/tools.png)

## Review the Datasets

### 🎯 3.1 Review the aircraft information graph schema

The aircraft information `aircraft` tenant graph has been pre-built with data sourced from Wikipedia (available under the Creative Commons Attribution-ShareAlike 4.0 License).

The following cell shows the implicit _schema_ for the entity-relationship tier of this graph. It shows, for example, that entity nodes of type `Manufacturer` are connected to `Aircraft` entity nodes using a relation with the name `BUILT`.

In [ ]:
NB_CLASSIC = True

from graphrag_toolkit.lexical_graph.visualisation import GraphNotebookVisualisation

v = GraphNotebookVisualisation(nb_classic=NB_CLASSIC)
v.display_schema(tenant_id='aircraft')

### 🎯 3.2 Review the aviation incident graph schema

The aviation accident `ntsb` tenant graph has been pre-built with data courtesy of the National Transportation Safety Board (NTSB).

The following cell shows the implict _schema_ for the entity-relationship tier of the this graph. It shows, for example, that entity nodes of type `Aircraft` are connected to `AircraftComponent` entity nodes using a relation with the name `HAS_COMPONENT`.

In [ ]:
NB_CLASSIC = True

from graphrag_toolkit.lexical_graph.visualisation import GraphNotebookVisualisation

v = GraphNotebookVisualisation(nb_classic=NB_CLASSIC)
v.display_schema(tenant_id='ntsb')

## Create an MCP Server and Client

### 🎯 3.3 Create an MCP server

The following cells create an MCP server that hosts a catalog of tools – one per tenant graph. The cells take a few seconds to run while the tool descriptions are auto-generated.

In [ ]:
import logging

from graphrag_toolkit.lexical_graph import set_advanced_logging_config

set_advanced_logging_config(
    logging_level=logging.DEBUG,
    included_modules={
        logging.DEBUG: [
            'graphrag_toolkit.lexical_graph.protocols', 
            'graphrag_toolkit.lexical_graph.retrieval.summary'
        ],
        logging.INFO: '*',
    },
    excluded_modules={
        logging.DEBUG: ['opensearch', 'boto', 'urllib'],
        logging.INFO: ['opensearch', 'boto', 'urllib', 'mcp', 'httpx'],
    }
)

In [ ]:
%reload_ext dotenv
%dotenv

import os

from graphrag_toolkit.lexical_graph.storage import GraphStoreFactory, VectorStoreFactory
from graphrag_toolkit.lexical_graph.protocols import create_mcp_server
from graphrag_toolkit.lexical_graph.retrieval.retrievers import *

graph_store = GraphStoreFactory.for_graph_store(os.environ['GRAPH_STORE'])
vector_store = VectorStoreFactory.for_vector_store(os.environ['VECTOR_STORE'])

tenant_config = {
    'aircraft': {},
    'ntsb': {
        'query_engine_args': {
            'retrievers': [ChunkBasedSemanticSearch, EntityBasedSearch, EntityNetworkSearch]
        }
    }
}

mcp_server = create_mcp_server(graph_store, vector_store, tenant_ids=tenant_config)

print('Server initialized')

### 🎯 3.4 Start the server

The cell below starts the MCP server.

In [ ]:
import threading

def run_server():
    mcp_server.run(transport='streamable-http', log_level='warning')
    
thread = threading.Thread(target=run_server)
thread.start()

### 🎯 3.5 Create an MCP client

[Strands Agents](https://strandsagents.com/latest/) is an open source SDK that takes a model-driven approach to building and running AI agents in just a few lines of code.

The cell below creates an MCP client that you can then use in a Strands Agent for answering cross-domain questions.

In [ ]:
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient

def create_streamable_http_transport():
    return streamablehttp_client('http://localhost:8000/mcp/')

mcp_client = MCPClient(create_streamable_http_transport)

### 🎯 3.6 Inspect the tool descriptions

The code below prints out the tool descriptions that have been auto-generated from the tenant graphs. Each tool is named after its tenant.

In [ ]:
with mcp_client:
    
    tools = mcp_client.list_tools_sync()
    
    for tool in tools:
        print(f"{tool.tool_spec['name']}: {tool.tool_spec['description']}")
        print('\n-------------------------------------\n')

## Run an Agentic Workload

### 🎯 3.7 Create an agent and ask some questions

You can now create a Strands AI Agent, and ask a question. The agent will choose the most appropriate tools for answering the question.

In [ ]:
%run './misc/strands_helpers.py'

from graphrag_toolkit.lexical_graph import GraphRAGConfig
from strands import Agent
        
with mcp_client:

    tools = mcp_client.list_tools_sync()
    agent = Agent(
        model=for_strands(GraphRAGConfig.response_llm),
        tools=tools,
        callback_handler=callback_handler(),
        system_prompt='''You are a helpful assistant. 
        Answer the user question based only on the evidence of the search results. 
        Reference information from the search results in your answer by adding the source information in square brackets at the end of relevant sentences.'''
    )
    
    
    agent("""Which Cessna 172 variants have been involved in accidents during instructional flights, 
    and how do their technical specifications and mechanical systems correlate with the common failure 
    points identified in accident reports?""")
    
    #agent("""What fuel system design features of the Piper PA-30 Twin Comanche contributed to fuel 
    #starvation accidents, and how did these compare to other twin-engine aircraft in the same class?""")
    
    #agent("""What safety issues and accident patterns do Kitfox series experimental aircraft demonstrate, 
    #and how do these compare to the design features and manufacturing specifications provided by Denney 
    #Aerocraft?""")

    #agent("""For accidents involving the Cessna 210 Centurion, what correlation exists between the 
    #aircraft's specific variants and particular types of mechanical failures or safety incidents?""")

    #agent("""How do accidents involving Robinson helicopters manufactured after 2000 compare to those 
    #involving Bell helicopters in terms of frequency and severity, and what specific design differences 
    #between these manufacturers' models might contribute to these accident patterns?""")
    
    #agent("""For aircraft equipped with Lycoming engines that were involved in accidents between 2010-2020, 
    #what were the technical specifications and known maintenance challenges of these engine types, and did 
    #the NTSB investigations identify any common failure patterns across different aircraft manufacturers 
    #using these engines?""")
 

### 🎯 3.8 Review tool invocations

The cell below prints the individual tool invocations – each `query` from the agent to a specific tool, and the `response` returned by the tool to the agent:

In [ ]:
import json

for i in agent.callback_handler.tool_invocations:
    print(json.dumps(i, indent=2))